# Bert 模型的简易使用说明
**这段时间又仔细学习了下bert的具体使用方法，这里做个小总结，只总结用法，不涉及原理，方便日后再查询**    
这里主要分为3部分，即：  
1. 模型的加载（实例化）
2. 模型保存
3. 模型的使用：tokenization, 前向传播

In [2]:
import torch
import torch.nn as nn
import transformers
from transformers import BertConfig, BertModel, BertTokenizer

In [3]:
pretrained_model_path = 'Bert_cn'
tuned_bert = 'Bert_cn/qa_tuned_bert.pth'
model_save_path = 'usage_path/'
tokenizer_path = 'Bert_cn/vocab.txt'

## 1. 加载预训练Bert模型

### 1.1 直接使用bert模型

In [4]:
# 直接加载预训练的中文bert模型
Model = BertModel.from_pretrained(pretrained_model_path)
# 当然也可以用自己微调好的模型替换参数
Model.load_state_dict(torch.load(tuned_bert))

Some weights of the model checkpoint at Bert_cn were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

### 1.2 使用bert作为我们模型的一部分

In [9]:
# 定义我们自己的模型
class MyModel(nn.Module):
    def __init__(self, pretrained_model: str, tuned_bert: str, pooling: str):
        super().__init__()
        # 先初始化bert
        self.query_bert = BertModel.from_pretrained(pretrained_model)
        # 再加载我之前微调的模型
        self.query_bert.load_state_dict(torch.load(tuned_bert))
        # 其余的初始化
        self.pooling = pooling

    def forward(self, sent):
        query_out = self.query_bert(sent[0], sent[1], sent[2], output_hidden_states=True)
        # 取哪一部分作为句向量？
        # cls：[CLS]这个token
        # pooler: [CLS]这个token再过一下全连接层+Tanh激活函数，作为该句子的特征向量。
        if self.pooling == 'cls':
            out = query_out[0][:, 0]  # [batch_size, 768]
        if self.pooling == 'pooler':
            out = query_out.pooler_output  # [batch_size, 768]
        return out

# 实例模型
Model = MyModel(pretrained_model_path, tuned_bert, 'cls')

Some weights of the model checkpoint at Bert_cn were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model init done!


## 2. 模型的保存
### 2.1 保存模型参数（推荐）

In [11]:
torch.save(Model.state_dict(), model_save_path+'model_1.pth')

### 2.2 保存整个模型（要占用内存，不推荐）

In [12]:
torch.save(Model, model_save_path+'model_2.pth')

### 2.3 保存模型的某一个部分
**例如这里我们保存之前MyModel里的‘query_bert’这一部分的参数**

In [13]:
torch.save(Model.query_bert.state_dict(), model_save_path+'model_3.pth')

## 3 模型的使用
### 3.1 tokenization
**首先，需要实例tokenizer:**

In [5]:
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1706: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


**然后处理我们的单个句子:**

In [6]:
sent = '你好，我是栗滕'
sent_source = tokenizer(sent,
                        max_length=15,
                        truncation=True,
                        padding='max_length',
                        return_tensors='pt')
# 得到bert模型需要的输入
input_ids = sent_source['input_ids']               # input_ids 是句子token化，id化之后的结果
attention_mask = sent_source['attention_mask']     # attention_mask 是mask标记，1表示没有被遮盖，0表示被遮盖（padding的数据会被遮盖）
token_type_ids = sent_source['token_type_ids']     # token_type_ids 是token类型标记，0表示第一句话，1表示第二句
print(input_ids)
print(attention_mask)
print(token_type_ids)

tensor([[ 101,  872, 1962, 8024, 2769, 3221, 3412, 4001,  102,    0,    0,    0,
            0,    0,    0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


**当然tokenizer还可以处理两个句子:**

In [12]:
sent1 = '你好陈清扬'
sent2 = '我是王二'
sent_source = tokenizer(sent1, sent2,
                        max_length=15,
                        truncation=True,
                        padding='max_length',
                        return_tensors='pt')
# 得到bert模型需要的输入
input_ids = sent_source['input_ids']               # input_ids 是句子token化，id化之后的结果，101是[CLS]，102是[SEP]
attention_mask = sent_source['attention_mask']     # attention_mask 是mask标记，1表示没有被遮盖，0表示被遮盖（padding的数据会被遮盖）
token_type_ids = sent_source['token_type_ids']     # token_type_ids 是token类型标记，0表示第一句话，1表示第二句
print(input_ids)
print(attention_mask)
print(token_type_ids)

tensor([[ 101,  872, 1962, 7357, 3926, 2813,  102, 2769, 3221, 4374,  753,  102,
            0,    0,    0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0]])


**tokenizer输入也可以是一个list的句子，你可以理解为一个batch的句子:**

In [13]:
sent_list = ['你好陈清扬', '我是王二', '蹲伟大的友谊！']
sent_source = tokenizer(sent_list,
                        max_length=15,
                        truncation=True,
                        padding='max_length',
                        return_tensors='pt')
# 得到bert模型需要的输入
input_ids = sent_source['input_ids']               # input_ids 是句子token化，id化之后的结果，101是[CLS]，102是[SEP]
attention_mask = sent_source['attention_mask']     # attention_mask 是mask标记，1表示没有被遮盖，0表示被遮盖（padding的数据会被遮盖）
token_type_ids = sent_source['token_type_ids']     # token_type_ids 是token类型标记，0表示第一句话，1表示第二句
print(input_ids)
print(attention_mask)
print(token_type_ids)

tensor([[ 101,  872, 1962, 7357, 3926, 2813,  102,    0,    0,    0,    0,    0,
            0,    0,    0],
        [ 101, 2769, 3221, 4374,  753,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0],
        [ 101, 6702,  836, 1920, 4638, 1351, 6449, 8013,  102,    0,    0,    0,
            0,    0,    0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


### 3.2 forward
**首先前向传播，得到模型输出(Out)**

In [19]:
Out = Model(input_ids, attention_mask, token_type_ids, output_hidden_states=True)

**然后我们来看看模型的输出都有哪些，以及对应的格式    
首先看输出的种类**

In [20]:
print(Out.keys())

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])


**可以发现，模型有3种，分别是last_hidden_state， pooler_output， hidden_states    
last_hidden_state：最后一层输出，形状是[batch_size, seq_lenth, 768]    
pooler: 常用来表示句向量本质上是‘[CLS]’这个token的tensor再过一下全连接层+Tanh激活函数，形状是 [batch_size, 768]    
hidden_states: 13层每层的输出，tuple，长度为13，每一层形状[batch_size, seq_lenth, 768]，它的最后一层也就是last_hidden_state    
而我们常常使用的cls，则是last_hidden_state的第一个向量，形状[batch_size, 768]**

In [34]:
# last_hidden_state
last_hidden_state_out = Out['last_hidden_state']
print('last_hidden_state_out形状:', last_hidden_state_out.shape)
# pooler
pooler_out = Out['pooler_output']
print('pooler_out形状:', pooler_out.shape)
# hidden_states, 
hidden_states_out = Out['hidden_states']
print('bert隐藏层层数:',len(hidden_states_out))
print('每层shape:',hidden_states_out[0].shape)
print('最后一层[-1]是否等于last_hidden_state:', hidden_states_out[-1]==last_hidden_state_out)
# cls
cls_out = Out['last_hidden_state'][:,0]
print('cls的形状：', cls_out.shape)

last_hidden_state_out形状: torch.Size([3, 15, 768])
pooler_out形状: torch.Size([3, 768])
bert隐藏层层数: 13
每层shape: torch.Size([3, 15, 768])
最后一层[-1]是否等于last_hidden_state: tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         

**同时我们可以知道，Out['last_hidden_state'] 和 Out[0] 是同一个数据 (同理，Out[1]代表pooler，Out[2]是hideen_states)**

In [26]:
print(Out['last_hidden_state'] == Out[0])
print(Out['pooler_output'] == Out[1])
print(Out['hidden_states'] == Out[2])

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]]